In [138]:
!nvidia-smi

Thu Mar 24 09:01:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    74W / 149W |   8604MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [139]:
! pip install transformers==4.0.1

In [140]:
!pip install torch==1.4.0

In [141]:
import torch
import random
import numpy as np
torch.cuda.is_available()

True

In [142]:
config = {
    'train_file_path':'/content/drive/MyDrive/Colab Notebooks/dataset/train.csv',
    'test_file_path':'/content/drive/MyDrive/Colab Notebooks/dataset/test.csv',
    'train_val_ratio':0.1,
    'model_path':'/content/drive/MyDrive/Colab Notebooks/dataset/BERT_model',
    'batch_size':16,
    'head': 'cnn',
    'num_epochs':1,
    'learning_rate':2e-5,
    'logging_step':500,
    'seed':2022
}
config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2022

## defaultdict
defaultdict的作用是在于，当字典里的key不存在但被查找时，返回的不是keyError而是一个默认值：

举例
dict1 = defaultdict(int)

dict2 = defaultdict(set)

dict3 = defaultdict(str)

dict4 = defaultdict(list)

print(dict1[1])，print(dict2[1])，print(dict3[1])，print(dict4[1])

0


set()

<br /> 

[]


## Python3.6 之 f'{}'
F字符串是开头有一个f的字符串文字，以及包含表达式的大括号将被其值替换。表达式在运行时进行渲染，然后使用__format__协议进行格式化。

举例：

name = "Eric"

age = 74

f"Hello, {name}. You are {age}."

'Hello, Eric. You are 74.'

## np.unique()

该函数是去除数组中的重复数字，并进行排序之后输出。


In [143]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
def read_data(config, tokenizer, mode = 'train'):
    data_df = pd.read_csv(config[f'{mode}_file_path'], sep=',')
    if mode == 'train':
        # 如果是训练文件 则划分训练集、验证集 【验证集(validation set)】
        X_train, y_train = defaultdict(list),[]
        X_val, y_val = defaultdict(list),[]
        num_val = int(len(data_df) * config['train_val_ratio'])
    else:
        #只生成测试集
        X_test, y_test = defaultdict(list),[]
        
        
    for i, row in tqdm(data_df.iterrows(), desc=f'preprocess {mode} data', total =len(data_df)):
        #得到每个句子的标签
        label = row[1] if mode == 'train' else 0
        #得到每个句子
        sentence = row[-1]
        # add_special_tokens 添加一些特殊的toke： CLS、 SEP
        # return_token_type_ids 返回每个词所对应的id
        # return_attention_mask 批量时， padding部分无需注意
        inputs = tokenizer.encode_plus(sentence, add_special_tokens = True, return_token_type_ids =True, return_attention_mask = True )
        #print("inputs:", inputs)
        #一个 inputs中:
        #inputs: {'input_ids': [101, 677, 6440, 3198, 2110, 4495, 2797, 3322, 1510, 702, 679, 977, 8024, 5439, 2360, 671, 2584, 722, 678, 2828, 2797, 3322, 3035, 749, 8024, 2157, 7270, 2897, 1355, 4873, 6375, 5439, 2360, 6608, 8024, 1920, 2157, 2582, 720, 4692, 2521, 6821, 4905, 752, 8043, 102], 
        #'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
        #'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
        # return：
        # input_ids；
        # token_type_ids
        # attention_mask
        if mode == 'train':
            if i < num_val:
                X_val['inputs_ids'].append(inputs['input_ids'])
                y_val.append(label)
                X_val['token_type_ids'].append(inputs['token_type_ids'])
                X_val['attention_mask'].append(inputs['attention_mask'])
            else:
                X_train['inputs_ids'].append(inputs['input_ids'])
                y_train.append(label)
                X_train['token_type_ids'].append(inputs['token_type_ids'])
                X_train['attention_mask'].append(inputs['attention_mask'])
                
        else:
                X_test['inputs_ids'].append(inputs['input_ids'])
                y_test.append(label) 
                X_test['token_type_ids'].append(inputs['token_type_ids'])
                X_test['attention_mask'].append(inputs['attention_mask'])
                
                
    if mode == 'train':
        label2id ={label: i for i, label in enumerate(np.unique(y_train))}
        #print("label2id",label2id)
        #label2id {100: 0, 101: 1, 102: 2, 103: 3, 104: 4, 106: 5, 107: 6, 108: 7, 109: 8, 110: 9, 112: 10, 113: 11, 114: 12, 115: 13, 116: 14}
        id2label ={i: label for label, i in label2id.items()}
        #print('id2label',id2label)
        #id2label {0: 100, 1: 101, 2: 102, 3: 103, 4: 104, 5: 106, 6: 107, 7: 108, 8: 109, 9: 110, 10: 112, 11: 113, 12: 114, 13: 115, 14: 116}
        y_train = torch.tensor([label2id[i] for i in y_train],dtype = torch.long)
        #print('y_train',y_train)
        #y_train tensor([ 3, 14,  6,  ...,  7,  6,  6])
        y_val = torch.tensor([label2id[i] for i in y_val],dtype = torch.long)
        return X_train, y_train, X_val, y_val, label2id, id2label
    
    else:
        y_test = torch.tensor(y_test, dtype = torch.long)
        return X_test,y_test        

## Dataset
* torch.utils.data.Dataset是代表自定义数据集方法的抽象类，你可以自己定义你的数据类继承这个抽象类，非常简单，只需要定义\_\_len\_\_和\_\_getitem\_\_ 这两个方法就可以。
* 通过继承torch.utils.data.Dataset的这个抽象类，我们可以定义好我们需要的数据类。当我们通过迭代的方式来取得每一个数据，但是这样很难实现取batch，shuffle或者多线程读取数据，所以pytorch还提供了一个简单的方法来做这件事情，通过torch.utils.data.DataLoader类来定义一个新的迭代器，用来将自定义的数据读取接口的输出或者PyTorch已有的数据读取接口的输入按照batch size封装成Tensor，后续只需要再包装成Variable即可作为模型的输入。

在torch.utils.data.Dataset和torch.utils.data.DataLoader这两个类中会用到python抽象类的魔法方法，包括\_\_len\_\_(self)，\_\_getitem\_\_(self)和\_\_iter\_\_(self)
* \_\_len\_\_(self) 定义当被len()函数调用时的行为（返回容器中元素的个数）
* \_\_getitem\_\_(self)定义获取容器中指定元素的行为，相当于self[key]，即允许类对象可以有索引操作。
* \_\_iter\_\_(self)定义当迭代容器中的元素的行为

## tensor size
行数：

tensor.size()[0]

or

tensor.size(0)

列数：

tensor.size()[1]

or

tensor.size(1)

举例：

a = torch.Tensor([[1,2,3],[4,5,6]])

s1 = a.size()[0]

s2 = a.size(0)

print(s1,'\n',s2)

输出： 

2 

2

In [144]:
from torch.utils.data import Dataset
class TNEWSData(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y 
    
    def __getitem__(self, idx):
        return {
            'inputs_ids': self.x['inputs_ids'][idx],
            'label': self.y[idx],
            'token_type_ids': self.x['token_type_ids'][idx],
            'attention_mask': self.x['attention_mask'][idx]
        }
    def __len__(self):
        #print("y:",self.y)
        #y: tensor([ 3, 14,  6,  ...,  7,  6,  6])
        #print('y.size:',self.y.size,'y.size(0):',self.y.size(0))
        #y.size: <built-in method size of Tensor object at 0x7fc76aaac140>(Tensor 对象的内置方法大小为 140494464794944) y.size(0): 48024
        return self.y.size(0)

从TNEWSData返回的一个example 
     
        {
            'inputs_ids' : self.x['inputs_ids'][idx],
            'label' : self.y[idx],
            'token_type_ids': self.x['token_type_ids'][idx],
            'attention_mask': self.x['attention_mask'][idx]
            
        }
        
        
取多次 examples，使用collate_fn整合到一起，变成一个tensor, 以便并行加载

In [145]:
def collate_fn(example):
    #从 TNEWData 返回的{}中，多了两个key：token_type_ids、attention_mask
    input_ids_list = []
    labels = []
    token_type_ids_list = []
    attention_mask_list = []
    
    for ex in example:
        input_ids_list.append(ex['inputs_ids'])
        labels.append(ex['label'])
        token_type_ids_list.append(ex['token_type_ids'])
        attention_mask_list.append(ex['attention_mask'])
    
    #对齐放入tensor
    max_len = max(len(input_ids) for input_ids in input_ids_list)
    # shape (len(labels), max_length)
    input_ids_tensor = torch.zeros((len(labels), max_len), dtype=torch.long)
    # torch.zeros_like:生成和括号内变量维度维度一致的全是零的内容
    token_type_ids_tensor = torch.zeros_like(input_ids_tensor)
    attention_mask_tensor = torch.zeros_like(input_ids_tensor)
    
    
    # 把列表中的数据放入tensor里
    for i, input_ids in enumerate(input_ids_list):
        #print('len(input_ids):',len(input_ids))
        #每次循环input_ids长度不同
        #len(input_ids): 18
        #len(input_ids): 30
        #len(input_ids): 21
        #len(input_ids): 27
        input_ids_tensor[i, :len(input_ids)] = torch.tensor(input_ids, dtype = torch.long)
        #print('input_ids_tensor',input_ids_tensor)
        token_type_ids_tensor[i, :len(input_ids)] = torch.tensor(token_type_ids_list[i], dtype= torch.long)
        attention_mask_tensor[i, :len(input_ids)] = torch.tensor(attention_mask_list[i], dtype= torch.long)
        
        
    return {
        'input_ids': input_ids_tensor,
        'labels': torch.tensor(labels, dtype = torch.long),
        'token_type_ids': token_type_ids_tensor,
        'attention_mask':  attention_mask_tensor
    }

input_ids_tensor：

tensor([[ 101, 2207, 3301, 1351, 4514,  749, 1372, 4344, 8024, 1316, 6158, 5439,
         2360, 6842, 1726, 8024, 2157, 7270,  671, 4692, 5010, 1139,  749, 1898,
          102,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 7674, 1947, 2791, 6587, 1164, 4372, 6632, 3341, 6632, 7770,  702,
          782, 6579, 2791, 3339, 3327, 4372, 3227, 5865,  678, 7360,  102,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 2398, 2128, 2544, 6134, 2658, 6399, 1166, 2825, 3318, 7309, 7959,
          157, 8882, 1744, 7354, 4993, 6612,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 3805, 2255, 1277, 2199, 2456, 2276, 2422, 2141, 7741, 2110, 3413,
         3173, 3413, 1277, 8013, 7564, 6369,  127, 3299, 2419, 1184, 2458, 2339,
         8013,  102,    0,    0,    0,    0,    0,    0],
        [ 101, 3342, 1283, 2079, 3198, 7392, 8126, 2399, 7028, 1726, 9312, 2864,
         1196, 8024, 1086, 2428,  680, 7942, 3856, 4197, 3022, 3440, 7652, 4028,
         1923, 1988, 8013,  102,    0,    0,    0,    0],
        [ 101, 4886, 4294, 1591, 6413, 3315, 4506, 8038, 7414, 7323, 1377, 3140,
          679, 6206, 5632, 1429,  924, 7730, 2844, 5661, 1372, 2897,  162, 1220,
         1213, 3341, 2773, 5885, 6832, 3616, 8043,  102],
        [ 101, 1306, 3694, 1344,  691, 1814, 2110, 3413, 1724, 2399, 5277, 1063,
         4408, 6958, 2760,  133,  517, 7028, 3941, 3765,  671, 3189, 3952, 6381,
          518,  135,  102,    0,    0,    0,    0,    0],
        [ 101, 6237, 3123, 1092, 2773, 3322, 5312, 7607,  704, 1744, 1378, 3968,
         5401, 1744, 1348, 6663, 1139, 3341,  100, 1353, 2190,  100,  749,  102,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 8109, 2399, 1059, 1744, 4636, 2487, 1814, 2356, 2408,  691, 3300,
         1126, 2429, 1814, 2356,  677, 3528, 8043,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101,  711,  862, 6821, 2429, 3189, 3315, 2207, 2270, 5543, 6375, 1059,
         4413, 5686, 3318, 5106,  692,  751, 4685, 1184, 2518, 8043,  102,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 2496, 4906, 3683, 4775,  678, 8424, 1146, 1400, 1071,  800,  782,
         6963, 6432,  749,  784,  720, 8043, 5687, 2472, 6849, 4994, 2828, 4281,
         1430, 2533, 1922, 1920,  749,  102,    0,    0],
        [ 101,  803, 2289, 1366, 3333, 3333, 3696, 4638, 2401, 4886, 4495, 3833,
          102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101,  100, 5698, 7937, 2458, 7305,  100, 5143, 1154, 4638, 1045, 4817,
         1168, 2419, 3221,  679, 3221, 3633, 4276, 1450, 8043,  102,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101,  671, 5663, 6963, 2523, 2208, 3300,  782, 6432, 4638, 1091,  868,
         2825, 2341,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])

## from_pretrained
一般建议直接使用from_pretrained()方法加载已经预训练好的模型或者参数。
* from_pretrained() let you instantiate a model/configuration/tokenizer from a pretrained version either provided by the library itself (currently 27 models are provided as listed here) or stored locally (or on a server) by the user


In [146]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader
def build_dataloader(config):
    #加载词表
    tokenizer = BertTokenizer.from_pretrained(config['model_path'])
    X_train, y_train, X_val, y_val, label2id, id2label = read_data(config, tokenizer, mode='train')
    X_test, y_test = read_data(config, tokenizer, mode='test')
    
    train_dataset = TNEWSData(X_train, y_train)
    val_dataset=TNEWSData(X_val, y_val)
    test_dataset = TNEWSData(X_test, y_test)
    
    train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=True, collate_fn=collate_fn)
    val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=False, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=False, collate_fn=collate_fn)
    
    
    return train_dataloader, val_dataloader, test_dataloader, id2label

In [147]:
train_dataloader, val_dataloader, test_dataloader, id2label = build_dataloader(config)

preprocess test data: 100%|██████████| 10000/10000 [00:05<00:00, 1720.57it/s]


In [148]:
for batch in train_dataloader:
    print(batch)
    break

{'input_ids': tensor([[  101,  6821,  3613,  7413,  2128,   782,  2218,   689,  3300,  2361,
          3307,   749,  8043,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,   898,  5101,  2434,  8038,  8711,  2199,   711,   689,  1218,
          2372,  3341,  2342,  1920,  2356,  1767,  4958,  7313,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  8137, 12675,  7721,  6852,  5664,  3683,   722,  1184,  3300,
          1525,   763,  2825,  3318,  4960,  4788,  8043,  1469,  8137,  8144,
          5143,  1154,  3683,  8024,  3221,  1059,  7481,  4638,   102,     0,
             0,     0],
        [  101,  4510,  4828,  2486,  2198,  8013,   808,  6205,  3175,  4156,
          7222,  8024,  7716,   836,  3209,  8038,   704,  1744, 11400,  1798,
          5661,  3678,  2825,  3318,  7566,  

In [149]:
from sklearn.metrics import f1_score
def evaluation(config, model, val_dataloader):
    model.eval()
    preds = []
    labels = []
    val_loss = 0.
    val_iterator = tqdm(val_dataloader, desc='Evaluation', total=len(val_dataloader))
    
    with torch.no_grad():
        for batch in val_iterator:
            labels.append(batch['labels'])
            batch = {item: value.to(config['device']) for item, value in batch.items()}
            loss, logits = model(**batch)[:2]
            
            val_loss += loss.item()
            preds.append(logits.argmax(dim = -1).detach().cpu())
            
    avg_val_loss = val_loss / len(val_dataloader)
    labels = torch.cat(labels, dim = 0).numpy()
    preds = torch.cat(preds, dim=0).numpy()
    f1 = f1_score(labels, preds, average='macro')
    return avg_val_loss, f1

In [150]:
# train BERT
from transformers import BertConfig, BertForSequenceClassification
from transformers import AdamW
from tqdm import trange

def train(config, id2label, train_dataloader, val_dataloader):
    #配置文件
    bert_config = BertConfig.from_pretrained(config['model_path'])
    bert_config.num_labels = len(id2label)
    model = BertForSequenceClassification.from_pretrained(config['model_path'], config = bert_config)
    

    #优化器
    optimizer = AdamW(model.parameters(), lr = config['learning_rate'])
    
    #放入GPU
    model.to(config['device'])
    epoch_iterator = trange(config['num_epochs'])
    global_steps = 0
    train_loss = 0.
    logging_loss =0.
    
    #跑了几个迭代
    for epoch in epoch_iterator:
        train_iterator = tqdm(train_dataloader, desc='Training', total=len(train_dataloader))
        model.train()
        for batch in train_iterator:
            #字典中的value送入GPU
            batch = {item: value.to(config['device']) for item, value in batch.items()}
                        
            #将字典作为关键字参数传递给python中函数
            loss = model(**batch)[0]
            
            #模型参数梯度设置为0
            model.zero_grad()
            
            #反向传播
            loss.backward()
            
            #更新参数
            optimizer.step()
            
            #叠加loss
            train_loss += loss.item()
            global_steps += 1
            
            if global_steps % config['logging_step'] == 0:
                print_train_loss = (train_loss - logging_loss) / config['logging_step']
                
                logging_loss = train_loss
                
                avg_val_loss, f1 = evaluation(config, model, val_dataloader)
                
                print_log = f'>>> training loss: {print_train_loss: .5f}, val loss: {avg_val_loss: .5f}, valid f1 score: {f1: .5f}'
                #print_log = f'>>> training loss: {print_train_loss:.4f}, valid loss: {avg_val_loss:.4f}, ' \
                 #           f'valid f1 score: {f1:.4f}'
                print(print_log)
                model.train()
    return model

In [151]:
model = train(config, id2label, train_dataloader, val_dataloader)

Some weights of the model checkpoint at /content/drive/MyDrive/Colab Notebooks/dataset/BERT_model were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

>>> training loss:  1.67589, val loss:  1.40506, valid f1 score:  0.49810




Training:  17%|█▋        | 501/3002 [02:49<4:19:54,  6.24s/it]

Training:  17%|█▋        | 502/3002 [02:49<3:04:54,  4.44s/it]

Training:  17%|█▋        | 503/3002 [02:49<2:13:12,  3.20s/it]

Training:  17%|█▋        | 504/3002 [02:50<1:36:27,  2.32s/it]

Training:  17%|█▋        | 505/3002 [02:50<1:11:22,  1.71s/it]

Training:  17%|█▋        | 506/3002 [02:50<53:07,  1.28s/it]  

Training:  17%|█▋        | 507/3002 [02:51<40:15,  1.03it/s]

Training:  17%|█▋        | 508/3002 [02:51<31:14,  1.33it/s]

Training:  17%|█▋        | 509/3002 [02:51<25:39,  1.62it/s]

Training:  17%|█▋        | 510/3002 [02:51<21:07,  1.97it/s]

Training:  17%|█▋        | 511/3002 [02:52<17:51,  2.32it/s]

Training:  17%|█▋        | 512/3002 [02:52<15:36,  2.66it/s]

Training:  17%|█▋        | 513/3002 [02:52<13:56,  2.98it/s]

Training:  17%|█▋        | 514/3002 [02:52<12:55,  3.21it/s]

Training:  17%|█▋        | 515/3002 [02:53<12:05,  3.43it/s]

Training:  17%|█▋        | 516/3002 [02:53<11:34,  3.58i

>>> training loss:  1.39974, val loss:  1.32729, valid f1 score:  0.51108




Training:  33%|███▎      | 1001/3002 [05:36<3:28:46,  6.26s/it]

Training:  33%|███▎      | 1002/3002 [05:36<2:28:33,  4.46s/it]

Training:  33%|███▎      | 1003/3002 [05:37<1:47:23,  3.22s/it]

Training:  33%|███▎      | 1004/3002 [05:37<1:18:19,  2.35s/it]

Training:  33%|███▎      | 1005/3002 [05:37<57:35,  1.73s/it]  

Training:  34%|███▎      | 1006/3002 [05:38<43:21,  1.30s/it]

Training:  34%|███▎      | 1007/3002 [05:38<33:19,  1.00s/it]

Training:  34%|███▎      | 1008/3002 [05:38<25:52,  1.28it/s]

Training:  34%|███▎      | 1009/3002 [05:38<20:36,  1.61it/s]

Training:  34%|███▎      | 1010/3002 [05:39<17:30,  1.90it/s]

Training:  34%|███▎      | 1011/3002 [05:39<15:23,  2.16it/s]

Training:  34%|███▎      | 1012/3002 [05:39<14:14,  2.33it/s]

Training:  34%|███▎      | 1013/3002 [05:40<12:27,  2.66it/s]

Training:  34%|███▍      | 1014/3002 [05:40<11:13,  2.95it/s]

Training:  34%|███▍      | 1015/3002 [05:40<10:40,  3.10it/s]

Training:  34%|███▍      | 1016/3002 [05:40

...



Training:  67%|██████▋   | 2001/3002 [11:09<1:44:02,  6.24s/it]

Training:  67%|██████▋   | 2002/3002 [11:10<1:14:07,  4.45s/it]

Training:  67%|██████▋   | 2003/3002 [11:10<53:22,  3.21s/it]  

Training:  67%|██████▋   | 2004/3002 [11:10<38:44,  2.33s/it]

Training:  67%|██████▋   | 2005/3002 [11:10<28:22,  1.71s/it]

Training:  67%|██████▋   | 2006/3002 [11:11<21:12,  1.28s/it]

Training:  67%|██████▋   | 2007/3002 [11:11<16:02,  1.03it/s]

Training:  67%|██████▋   | 2008/3002 [11:11<12:29,  1.33it/s]

Training:  67%|██████▋   | 2009/3002 [11:11<10:08,  1.63it/s]

Training:  67%|██████▋   | 2010/3002 [11:12<08:33,  1.93it/s]

Training:  67%|██████▋   | 2011/3002 [11:12<07:22,  2.24it/s]

Training:  67%|██████▋   | 2012/3002 [11:12<06:34,  2.51it/s]

Training:  67%|██████▋   | 2013/3002 [11:13<05:58,  2.76it/s]

Training:  67%|██████▋   | 2014/3002 [11:13<05:30,  2.99it/s]

Training:  67%|██████▋   | 2015/3002 [11:13<05:10,  3.18it/s]

Training:  67%|██████▋   | 2016/3002 [11:13<04:

>>> training loss:  1.30153, val loss:  1.27490, valid f1 score:  0.53048




Training:  83%|████████▎ | 2501/3002 [13:55<52:17,  6.26s/it]  

Training:  83%|████████▎ | 2502/3002 [13:56<37:09,  4.46s/it]

Training:  83%|████████▎ | 2503/3002 [13:56<26:35,  3.20s/it]

Training:  83%|████████▎ | 2504/3002 [13:56<19:19,  2.33s/it]

Training:  83%|████████▎ | 2505/3002 [13:56<14:09,  1.71s/it]

Training:  83%|████████▎ | 2506/3002 [13:57<10:39,  1.29s/it]

Training:  84%|████████▎ | 2507/3002 [13:57<08:04,  1.02it/s]

Training:  84%|████████▎ | 2508/3002 [13:57<06:24,  1.28it/s]

Training:  84%|████████▎ | 2509/3002 [13:57<05:06,  1.61it/s]

Training:  84%|████████▎ | 2510/3002 [13:58<04:14,  1.93it/s]

Training:  84%|████████▎ | 2511/3002 [13:58<03:38,  2.25it/s]

Training:  84%|████████▎ | 2512/3002 [13:58<03:17,  2.48it/s]

Training:  84%|████████▎ | 2513/3002 [13:59<02:59,  2.72it/s]

Training:  84%|████████▎ | 2514/3002 [13:59<02:42,  3.00it/s]

Training:  84%|████████▍ | 2515/3002 [13:59<02:35,  3.13it/s]

Training:  84%|████████▍ | 2516/3002 [13:59<02:26, 

>>> training loss:  1.28893, val loss:  1.25458, valid f1 score:  0.52962




Training: 100%|█████████▉| 3001/3002 [16:42<00:06,  6.27s/it]

100%|██████████| 1/1 [16:42<00:00, 1002.72s/it]


In [152]:
def predict(config, id2label, model, test_dataloader):
    test_iterator = tqdm(test_dataloader, desc='Predicting', total=len(test_dataloader))
    model.eval()
    test_preds = []
    with torch.no_grad():
        for batch in test_iterator:
            batch = {item: value.to(config['device']) for item, value in batch.items()}
            logits = model(**batch)[1]

            test_preds.append(logits.argmax(dim=-1).detach().cpu())
    test_preds = torch.cat(test_preds, dim=0).numpy()
    test_preds = [id2label[id_] for id_ in test_preds]
    test_df = pd.read_csv(config['test_file_path'], sep=',')
    test_df.insert(1, column='label', value=test_preds)
    test_df.drop(columns=['sentence'], inplace=True)
    test_df.to_csv('submission.csv', index=False, encoding='utf8')

In [153]:
predict(config, id2label, model, test_dataloader)

Predicting: 100%|██████████| 625/625 [00:49<00:00, 12.58it/s]
